# COGS 188 Lab 5 Part 3: Traveling Salesman Problem (TSP) with Dynamic Programming

## Problem Description
In a previous lab, you learned how to solve the TSP by modeling it as a constrain-satisfaction problem (CSP). This time, you'll tackle it using dynamic programming.

Consider a set of cities and the cost to travel between each pair of cities. The goal is to find the shortest path that visits each city exactly once and returns to the starting city. In this case, the "cost" of traveling between cities is the Euclidean distance between them.

## Mathematical Definitions

1. **State Space:**
   - A state $s$ is represented as a tuple:
     - $s = (c, V)$
     - $c$: Current city.
     - $V$: Set of visited cities represented as a bitmask.
   - Example state: $(2, 0\text{b}1011)$, where:
     - $2$ is the index of the current city.
     - $0\text{b}1011$ represents the cities visited so far, which means that cities $0$, $1$, and $3$ have been visited and city $2$ has not been visited yet.

2. **Action Space:**
   - An action $a$ is the index of the next city to visit.
   - Possible actions from a state $s$ are all cities not yet visited in $V$.
   - $a \in \{0, 1, 2, 3\} \setminus V$

3. **Transition Function:**
   - The transition function $T$ defines the next state given a state-action pair:
   $$T(s, a) = (a, V \cup \{a\})$$

   where:
   - $s = (c, V)$ is the current state.
   - $a$ is the next city to visit.
   - $V \cup \{a\}$ is the updated set of visited cities.

4. **Reward Function:**
   - The reward function $R$ defines the immediate cost of traveling from the current city to the next city:
   $$R(s, a) = -d(c, a)$$
   
   where $d(c, a)$ is the distance between cities $c$ and $a$, where $c$ is the current city.

5. **Goal:**
   - The goal is to find the optimal policy $\pi^*$ that minimizes the total travel cost:
  
   $$\pi^* = \arg\min_{\pi} \mathbb{E} \left[ \sum_{t=0}^{T} R(s_t, a_t) \right]$$
  
   (For simplicity, we'll set the discount factor $\gamma = 1$.)

   where:
   - $s_t$: State at time step $t$.
   - $a_t$: Action at time step $t$.
   - $T$ is the planning horizon, which is the total number of cities in the problem.

6. **Value Function:**
   - The value function $V$ represents the minimum travel cost from a state $s$:
   $$V(s) = \min_{a} \left[ R(s, a) + V(T(s, a)) \right]$$

7. **Q-Matrix:**
   - The Q-matrix represents the expected cost for each state-action pair:
   $$Q(s, a) = R(s, a) + V(T(s, a))$$

In [ ]:
# Import the necessary packages
import numpy as np
import matplotlib.pyplot as plt

Firstly, we will generate a random map of cities and their coordinates. You may play with the variable `n_cities` to generate maps with a different number of cities to test your implementation. I recommend you first try it with around 6-10 cities before moving to larger maps.

In [ ]:
np.random.seed(42)

n_cities = 20  # You can change this value to increase or decrease the number of cities

# Generate random coordinates for the cities
cities = {f'City_{i}': (np.random.uniform(0, 100), np.random.uniform(0, 100)) for i in range(n_cities)}

Then, we will create a **distance matrix**, which will be used to store the Euclidean distances between every pair of cities. The distance matrix is a 2D array where `dist_matrix[i][j]` represents the distance between city `i` and city `j`. This matrix is symmetric, because `dist_matrix[i][j]` is equal to `dist_matrix[j][i]`. Furthermore, the diagonal elements are set to zero, because the distance between a city and itself is zero.

In [ ]:
# Function to calculate the Euclidean distance between two cities
def calculate_distance(city1, city2):
    return np.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Create a distance matrix for the cities
city_names = list(cities.keys())
distance_matrix = np.zeros((n_cities, n_cities))

for i, city1 in enumerate(city_names):
    for j, city2 in enumerate(city_names):
        distance_matrix[i, j] = calculate_distance(cities[city1], cities[city2])

# Plot the cities on a map
plt.figure(figsize=(8, 8))
plt.scatter(*zip(*cities.values()), marker='o', color='red')
for name, (x, y) in cities.items():
    plt.text(x, y, f" {name}", fontsize=12)
plt.xlabel("X Coordinate")
plt.ylabel("Y Coordinate")
plt.title("Randomly Generated Cities for TSP")
plt.grid(True)
plt.show()

print("Distance Matrix:")
print(distance_matrix)

### Task Instructions:

1. **Implement the `tsp_dynamic_programming` function:**
   - Your goal is to implement the dynamic programming algorithm that solves the TSP.
   - The function signature and expected return values are provided below.

In [ ]:
# TODO: Implement the Dynamic Programming Solution for TSP
def tsp_dynamic_programming(distance_matrix):
    """
    Solve the Traveling Salesman Problem using dynamic programming.

    Parameters:
    - distance_matrix (np.ndarray): Matrix of distances between cities, that we defined earlier

    Returns:
    - path (list): Optimal path visiting all cities.
        - this is a list of strings representing the city names
    - min_cost (float): Minimum travel cost.
    - value_function (dict): The value function for all states.
        - this is a dictionary with keys as (city, visited) and values as the minimum cost to visit all cities.
    - q_matrix (dict): Q-matrix representing action values for each state.
        - this is a dictionary with keys as (city, visited) and values as a dictionary of next city and cost.
    """
    n_cities = distance_matrix.shape[0]
    # Initialize memoization table: this table stores the minimum cost to visit each city
    # 2**n_cities are the possible sets of visited cities (represented as bitmasks)
    memo = [[None] * (2**n_cities) for _ in range(n_cities)]
    # Initialize parent table to reconstruct the optimal path
    # The parent table stores the previous city to visit for each state
    parent = [[None] * (2**n_cities) for _ in range(n_cities)]
    # Initialize dictionaries to store the value function and Q-matrix
    value_function = {}
    q_matrix = {}

    def tsp_recursive(city, visited):
        """
        Recursive function to solve TSP using dynamic programming.

        Parameters:
        - city (int): Index of the current city.
        - visited (int): Bitmask representing the set of visited cities.

        Returns:
        - float: Minimum travel cost to complete the tour from the current state.
        """
        # Base case: all cities visited, return to the starting city
        # TODO: Implement the base case to return the distance back to the starting city
        # visited is an integer, so you might want to convert it to binary to understand which cities have been visited

        # Check if the subproblem has already been solved
        # TODO: Return the cached result if memoization table has a value

        # Initialize the minimum cost and action values
        min_cost = float('inf')
        q_values = []

        # Iterate through all cities to find the next city to visit
        # TODO: Implement the loop to calculate costs for each possible next city


        # Cache the result in the memoization table
        memo[city][visited] = min_cost

        # Store the value function and Q-matrix
        value_function[(city, visited)] = min_cost
        q_matrix[(city, visited)] = {next_city: cost for next_city, cost in q_values}

        return min_cost

    # Start solving the TSP from the first city (index 0) with only the starting city visited
    # TODO: Call the recursive function with appropriate arguments to find the minimum cost
    min_cost = ...

    # Reconstruct the optimal path by following the parent table
    path = []
    visited = 1
    current_city = 0
    while current_city is not None:
        path.append(current_city)
        next_city = parent[current_city][visited]
        if next_city is None:
            break
        visited |= 1 << next_city  # Mark the next city as visited
        current_city = next_city

    path.append(0)  # Return to the starting city

    return path, min_cost, value_function, q_matrix

In [ ]:
# Test the Dynamic Programming solution
path_dp, min_cost_dp, value_function_dp, q_matrix_dp = tsp_dynamic_programming(distance_matrix)

# Display the results
print("Optimal Path (DP):", [city_names[i] for i in path_dp])
print("Minimum Cost (DP):", min_cost_dp)

Once you have implemented the `tsp_dynamic_programming` function, run the cell below to test your implementation.

In [ ]:
# Function to visualize the optimal TSP path
def plot_tsp_path(cities, path, city_names):
    """
    Plot the optimal TSP path.

    Parameters:
    - cities (dict): Dictionary of cities and their coordinates.
    - path (list): Optimal path visiting all cities.
    - city_names (list): List of city names in the correct order.
    """
    path_coordinates = [cities[city_names[i]] for i in path]
    path_coordinates.append(cities[city_names[path[0]]])  # Complete the tour

    plt.figure(figsize=(8, 8))
    plt.plot(*zip(*path_coordinates), marker='o', color='blue')
    for name, (x, y) in cities.items():
        plt.text(x, y, f" {name}", fontsize=12)
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.title("Optimal TSP Path")
    plt.grid(True)
    plt.savefig('tsp_path.png')
    plt.show()

# Plot the optimal path
plot_tsp_path(cities, path_dp, city_names)

## Submission Instructions

Before you turn in the notebook, run all the cells with **n_cities = 20** cities to make sure your solution runs as expected. The resulting figure should be saved to `tsp_path.png`.